### Descriptive Data

#### Heat map of crime with arrest in Chicago

In [5]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns
import censusdata
import numpy as np

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [6]:
crime = pd.read_csv("Data/Crimes_-_2019.csv")

In [9]:
neighborhood = gpd.read_file("Data/Boundaries - Neighborhoods.geojson")
crime = crime.dropna()
crs = {"init":"epsg:4326"}
crime = gpd.GeoDataFrame(crime, crs = crs, geometry=gpd.points_from_xy(crime.Longitude, crime.Latitude))
crime = gpd.sjoin(crime, neighborhood, how="inner", op='intersects')

In [10]:
crime_truncated = crime[crime.Arrest == True]
crime_truncated.shape

(54807, 28)

In [11]:
# Randomly select 5000 cases for visualization
crime_truncated = crime_truncated.sample(5000)

m = folium.Map([41.8781, -87.6298], zoom_start=11)
for index, row in crime_truncated.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius=0.3,
                        fill_color="#D3D3D3",
                       ).add_to(m)
stationArr = crime_truncated[['Latitude', 'Longitude']].to_numpy()

# plot heatmap
m.add_children(plugins.HeatMap(stationArr, radius=15))
m